In [1]:
import os
os.getcwd()

'c:\\Users\\Kun\\Desktop\\slide-classification\\code'

In [200]:
from typing import List
# from docarray import dataclass, Document, DocumentArray
# from docarray.typing import Image, Text, JSON, URI
import pandas as pd
import numpy as np
import ast
import platform
import re

is_windows = platform.system().lower() == 'windows'

if is_windows:
    source_dir = "C:\\Users\\Kun\\Desktop" 
else:
    source_dir = "/Users/kun-lin/Desktop"

source_dir = os.path.join(source_dir, f"Angebot")
dest_dir = "output"
PPTtoImg_dir = os.path.join(dest_dir, f"PPTtoImg")
fWordImg_dir = os.path.join(dest_dir, f"FewWordImg")
ct_fWordImg_dir = os.path.join(fWordImg_dir, f"CT_FewWordImg")
topshapetype_dir = os.path.join(dest_dir, f"TopShapeTypeImg")
labels_dir = os.path.join(dest_dir, f"LABELS")
checkpoint_path = os.path.join(dest_dir, f"checkpoints")

text_path = 'alignment_pptx_list.csv'

def convert_to_list(data):
    return ast.literal_eval(data)
# Since it take too much time on traversal ppt files, Easily to import the dataframe from the csv file
df_da = pd.read_csv(text_path, converters={'shape_type': convert_to_list})

df_da = df_da[~df_da['contents'].isnull()]
df_da.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26468 entries, 0 to 26467
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   26468 non-null  object
 1   page_num    26468 non-null  int64 
 2   shape_type  26468 non-null  object
 3   contents    26468 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [201]:
# filter out less than 4 words
df_da['word_count'] = df_da['contents'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
df_few_words = df_da[~(df_da['word_count'] <= 4)]
df_few_words.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23148 entries, 0 to 26467
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   23148 non-null  object
 1   page_num    23148 non-null  int64 
 2   shape_type  23148 non-null  object
 3   contents    23148 non-null  object
 4   word_count  23148 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


In [192]:
uu = '2564'
uu = 'Angebot\\' + uu
uu

'Angebot\\2564'

In [193]:
df_json = pd.read_json(os.path.join(labels_dir, "backup_23148_3558.json"))
# have done for annotations
df_json = df_json[df_json['annotations'].apply(lambda x: len(x) != 0)]
# annotation results exist
df_json = df_json[df_json['annotations'].apply(lambda x: len(x[0]['result']) != 0)] 
# 22348 for 13th slide
count = 13

df_few_words['label'] = ''
cccc = 0
for idx, row in enumerate(df_json.iterrows()):
  # if idx == 0:
  slide = row[1]
  image_path = slide['data']['image']
  choice = slide['annotations'][0]['result'][0]['value']['choices'][0]
  # print(file_path, choice)
  if image_path.find('22348')!=-1:
    # print(image_path, choice)
    new_image_name = image_path.split('.')[0] + '_' + str(count) + '.PNG'
    slide['data']['image'] = new_image_name
    image_path = new_image_name
    print(image_path)
    count += 1

  # get page number
  if image_path.split('.')[0].split('_')[-1].isdigit(): 
    page = int(image_path.split('.')[0].split('_')[-1])
  else:
    page = 0
    
  ktr = image_path.split('-')[1].split('_')[0]
  ktr = 'Angebot\\'+ ktr
  ktr = re.escape(ktr)
  # some issues that it should modify manually
  if (ktr == '22348' and page == 13):
    df_few_words.loc[(df_few_words['file_name'].str.contains(ktr)) & (df_few_words['page_num'] == page), 'label'] = 'Initial Situation'
    continue
  elif (ktr == '22348' and page == 14):
    df_few_words.loc[(df_few_words['file_name'].str.contains(ktr)) & (df_few_words['page_num'] == page), 'label'] = 'Target Situation'
    continue

  if(ktr=='21993' and page == 2):
    print(ktr, page, choice)

  df_few_words.loc[(df_few_words['file_name'].str.contains(ktr)) & (df_few_words['page_num'] == page), 'label'] = choice

print(cccc)
df_few_words.info()

C:\Users\Kun\AppData\Local\Temp\ipykernel_33636\2526150889.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_few_words['label'] = ''


/data/upload/3/30b5074c-22348_Projektskizze_Benchmarking_und_Effizienzsteigerung_in_den_Zentra_EVP8fho_13.PNG
/data/upload/3/33344dac-22348_Projektskizze_Benchmarking_und_Effizienzsteigerung_in_den_Zentra_iHJAu6y_14.PNG
0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23148 entries, 0 to 26467
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   23148 non-null  object
 1   page_num    23148 non-null  int64 
 2   shape_type  23148 non-null  object
 3   contents    23148 non-null  object
 4   word_count  23148 non-null  int64 
 5   label       23148 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [202]:
df_json = pd.read_json(os.path.join(labels_dir, "3557_annotations.json"))
df_json['choice'].value_counts()

Others                         577
Working Package Examples       542
Working Package Description    478
Reference Details              323
Offer Title                    284
Consultant Profile             265
Project Calculation            243
Initial & Target Situation     197
Working Package Overview       190
Reference Overview             168
Competencies                   147
Target Situation                79
Initial Situation               63
Name: choice, dtype: int64

In [203]:
for idx, row in df_json[df_json['image'].str.contains('22348')].iterrows():
  print(idx, row['image'])


165 /data/upload/3/33344dac-22348_Projektskizze_Benchmarking_und_Effizienzsteigerung_in_den_Zentra_iHJAu6y.PNG
2101 /data/upload/3/30b5074c-22348_Projektskizze_Benchmarking_und_Effizienzsteigerung_in_den_Zentra_EVP8fho.PNG


In [204]:
# deal with long file name which is not shown page number
count = 13
for idx, row in df_json[df_json['image'].str.contains('22348')].iterrows():
  new_image_name = df_json.loc[idx, 'image'].split('.')[0] + '_' + str(count) + '.PNG'
  print(idx, new_image_name)
  df_json.at[idx, 'image'] = new_image_name
  count += 1

165 /data/upload/3/33344dac-22348_Projektskizze_Benchmarking_und_Effizienzsteigerung_in_den_Zentra_iHJAu6y_13.PNG
2101 /data/upload/3/30b5074c-22348_Projektskizze_Benchmarking_und_Effizienzsteigerung_in_den_Zentra_EVP8fho_14.PNG


In [205]:
df_json['page_num'] = df_json['image'].apply(lambda x: int(x.split('.')[0].split('_')[-1]) if x.split('.')[0].split('_')[-1].isdigit() else 0)
df_json['image'] = df_json['image'].apply(lambda x: x.split('.')[0].split('-')[1])

In [206]:
# be aware of path difference between Windows and Mac OS
df_few_words['label'] = ''

for i in range(len(df_json)):
    item = df_json.iloc[i]
    f_name = item['image']
    page = int(df_json.at[i, 'page_num'])
    ktr = f_name.split('_')[0]

    # some issues that it should modify manually
    if (ktr == '22348' and page == 13):
      df_few_words.loc[(df_few_words['file_name'].str.contains(ktr)), 'label'] = 'Initial Situation'
      continue
    elif (ktr == '22348' and page == 14):
      df_few_words.loc[(df_few_words['file_name'].str.contains(ktr)), 'label'] = 'Target Situation'
      continue

    df_few_words.loc[(df_few_words['file_name'].str.contains(ktr)) & (df_few_words['page_num'] == page), 'label'] = item['choice']

df_few_words.info()

C:\Users\Kun\AppData\Local\Temp\ipykernel_33636\2693745292.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_few_words['label'] = ''


<class 'pandas.core.frame.DataFrame'>
Int64Index: 23148 entries, 0 to 26467
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   23148 non-null  object
 1   page_num    23148 non-null  int64 
 2   shape_type  23148 non-null  object
 3   contents    23148 non-null  object
 4   word_count  23148 non-null  int64 
 5   label       23147 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [97]:
df_few_words[df_few_words['label'].isna()]

,file_name,page_num,shape_type,contents,word_count,label


In [194]:
df_few_words['label'].value_counts()

                               19585
Others                           575
Working Package Examples         544
Working Package Description      478
Reference Details                324
Offer Title                      284
Consultant Profile               265
Project Calculation              244
Initial & Target Situation       197
Working Package Overview         194
Reference Overview               168
Competencies                     148
Target Situation                  79
Initial Situation                 63
Name: label, dtype: int64

# Advanced model construction

In [ ]:
from transformers import BertTokenizer, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

# class Dataset(torch.utils.data.Dataset):

#     def __init__(self, df):

#         self.labels = [labels[label] for label in df_few_words['category']]
#         self.texts = [tokenizer(text,
#                                 padding='max_length', max_length = 128, truncation=True,
#                                 return_tensors="pt") for text in df_few_words['contents']]

#     def classes(self):
#         return self.labels

#     def __len__(self):
#         return len(self.labels)

#     def get_batch_labels(self, idx):
#         # Fetch a batch of labels
#         return np.array(self.labels[idx])

#     def get_batch_texts(self, idx):
#         # Fetch a batch of inputs
#         return self.texts[idx]

#     def __getitem__(self, idx):

#         batch_texts = self.get_batch_texts(idx)
#         batch_y = self.get_batch_labels(idx)

#         return batch_texts, batch_y

In [207]:
df = df_few_words[(df_few_words['label']!='') & (df_few_words['label']!='Others')]

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3032 entries, 0 to 26467
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   3032 non-null   object
 1   page_num    3032 non-null   int64 
 2   shape_type  3032 non-null   object
 3   contents    3032 non-null   object
 4   word_count  3032 non-null   int64 
 5   label       3031 non-null   object
dtypes: int64(2), object(4)
memory usage: 165.8+ KB


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2988 entries, 0 to 26467
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   file_name   2988 non-null   object
 1   page_num    2988 non-null   int64 
 2   shape_type  2988 non-null   object
 3   contents    2988 non-null   object
 4   word_count  2988 non-null   int64 
 5   label       2988 non-null   object
dtypes: int64(2), object(4)
memory usage: 163.4+ KB


In [196]:
for i in df[df['label']=='Target Situation'].iterrows():
    print(i[1]['file_name'], i[1]['page_num'])

C:\Users\Kun\Desktop\Angebot\11580\Franz_Haniel_Akquisition_Process_V_0_1.pptx 12
C:\Users\Kun\Desktop\Angebot\11637\Anlage_1_Leistungsbeschreibung_11637.pptx 11
C:\Users\Kun\Desktop\Angebot\11815\Anlage 1 Leistungsbeschreibung zu Angebot Nr. 11815.pptx 11
C:\Users\Kun\Desktop\Angebot\12316\Enclosure 1 – Service Description to offer No. 12316.pptx 10
C:\Users\Kun\Desktop\Angebot\12355\Anlage_1_Angebot_12355.pptx 14
C:\Users\Kun\Desktop\Angebot\12449\Anlage1_12449.pptx 9
C:\Users\Kun\Desktop\Angebot\12649\20220711_UNITY_Anlage SBE.pptx 8
C:\Users\Kun\Desktop\Angebot\12653\Anlage_1_Leistungsbeschreibung_AP4_5_UKK.pptx 13
C:\Users\Kun\Desktop\Angebot\12675\Anlage_1_Leistungsbeschreibung_12675.pptx 5
C:\Users\Kun\Desktop\Angebot\12932\Anlage_1_Leistungsbeschreibung_12932_final.pptx 5
C:\Users\Kun\Desktop\Angebot\12966\Ergebnisdokumentation_Begehung.pptx 8
C:\Users\Kun\Desktop\Angebot\12968\Anlage_1_Leistungsbeschreibung_KTR12968.pptx 5
C:\Users\Kun\Desktop\Angebot\13106\Anlage_1_Leistungsb

In [ ]:
# split texts into several chunks
def get_split(text1):
  l_total = []
  l_parcial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150
  else:
    n = 1

  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:400]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*150:w*150 + 400]
      l_total.append(" ".join(l_parcial))
  return l_total

In [ ]:
labels = {'Competencies': 0,
            'Consultant Profile':1,
            'Initial & Target Situation':2,
            'Initial Situation':3,
            'Offer Title':4,
            'Project Calculation':5,
            'Reference Details':6,
            'Reference Overview':7,
            'Target Situation':8,
            'Working Package Description':9,
            'Working Package Examples':10,
            'Working Package Overview':11,
  }

df['label_encode'] = df['label'].apply(lambda x : labels[x])
df['split_contents'] = df['contents'].apply(lambda x :get_split(x))
df = df.explode('split_contents', ignore_index = True)
df.drop(columns = ['word_count'], inplace = True)

In [ ]:
contents = df.split_contents.values
# contents = df.contents.values
labels = df.label_encode.values

In [ ]:
# Print the original sentence.
print('Original: ', contents[0])
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(contents[0]))
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(contents[0])))

In [ ]:
# Measure the maximum sentence length.
max_len = 0

# For every sentence...
for sent in contents:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

MAX_LENGTH_SEQ = 400

# For every sentence...
for sent in contents:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                           # Sentence to encode.
                        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LENGTH_SEQ,    # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',          # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', contents[0])
print('Token IDs:', input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
)

# Train Our Classification Model

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 12, # The number of output labels--12 for multi-class classification.

        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 1.3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
import random
import numpy as np
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./output/runs/a_text_e5_b32_s512")

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(epochs):
    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    for step, batch in enumerate(train_dataloader):

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()


        if step % 40 == 39: # every 40 mini-batches....
          # Calculate elapsed time in minutes.
          elapsed = format_time(time.time() - t0)
          # Report progress.
          print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # ========================================
          #               Validation
          # ========================================
          # After the completion of each training epoch, measure our performance on
          # our validation set.
          print("")
          print(f"Running Validation...({step // 40})")
          
          t1 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model.eval()

          # Tracking variables
          total_eval_accuracy = 0
          total_eval_loss = 0
          predictions = []
          true_labels = []

          # Evaluate data for one epoch
          for batch in validation_dataloader:

              # Unpack this training batch from our dataloader.
              #
              # As we unpack the batch, we'll also copy each tensor to the GPU using
              # the `to` method.
              #
              # `batch` contains three pytorch tensors:
              #   [0]: input ids
              #   [1]: attention masks
              #   [2]: labels
              b_input_ids = batch[0].to(device)
              b_input_mask = batch[1].to(device)
              b_labels = batch[2].to(device)

              # Tell pytorch not to bother with constructing the compute graph during
              # the forward pass, since this is only needed for backprop (training).
              with torch.no_grad():

                  # Forward pass, calculate logit predictions.
                  # token_type_ids is the same as the "segment ids", which
                  # differentiates sentence 1 and 2 in 2-sentence tasks.
                  # The documentation for this `model` function is here:
                  # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                  # Get the "logits" output by the model. The "logits" are the output
                  # values prior to applying an activation function like the softmax.

                  outputs = model(b_input_ids,
                                  token_type_ids=None,
                                  attention_mask=b_input_mask,
                                  labels=b_labels)

                  loss = outputs.loss
                  logits = outputs.logits

              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.detach().cpu().numpy()
              label_ids = b_labels.to('cpu').numpy()

              # Accumulate the predictions and true labels
              predictions.extend(np.argmax(logits, axis=1))
              true_labels.extend(label_ids)
              
              # Calculate the accuracy for this batch of test sentences, and
              # accumulate it over all batches.
              total_eval_accuracy += flat_accuracy(logits, label_ids)



          # Calculate the average loss over mini-batches.
          avg_train_loss = total_train_loss / 40

          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(validation_dataloader)


          # Report the final accuracy for this validation run.
          avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
          print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t1)

          print("  Validation Loss: {0:.2f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          # Log the running loss averaged per batch
          writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_train_loss, 'Validation' : avg_val_loss },
                                epoch_i * len(train_dataloader) + step)

          total_train_loss = 0.0



    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    # print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))


print('Finished Training')
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
writer.flush()

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('display.precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
# % matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks(range(1,epochs))

plt.show()

In [ ]:
os.path.abspath(checkpoint_path)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir='c:/Users/Kun/Desktop/slide-classification/code/output/checkpoints' --port=6

In [ ]:
from sklearn.metrics import classification_report
# Convert the predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Generate the classification report
report = classification_report(true_labels, predictions, target_names=['Competencies[0]', 
                                                                            'Consultant Profile[1]', 
                                                                            'Initial & Target Situation[2]', 
                                                                            'Initial Situation[3]', 
                                                                            'Offer Title[4]', 
                                                                            'Project Calculation[5]', 
                                                                            'Reference Details[6]', 
                                                                            'Reference Overview[7]',
                                                                            'Target Situation[8]',
                                                                            'Working Package Description[9]',
                                                                            'Working Package Examples[10]',
                                                                            'Working Package Overview[11]'])

# Print the classification report
print(report)